# [Adaptive numerical integration][quarteroni]

[quarteroni]: https://www.amazon.it/Matematica-Numerica-Unitext-Quarteroni/dp/8847000777  "Reference book (in italian)"


In this project, we apply numerical methods to approximate a definite integral of the type: $I(f) = \int_a^b f(x)\, dx$, a process also called *numerical integration*.

We are going to apply the Cavalieri-Simpson integration method on subintervals $[\alpha, \beta]$ of the entire integration range $[a,b]$, with $[\alpha, \beta] \subseteq [a,b]$.

## Cavalieri-Simpson rule

The Cavalieri-Simpson rule approximates a definite integral of a function $f$ on an interval $[\alpha, \beta]$ by calculating the definite integral of the polynomial of the second order $P$ (a quadratic polynomial, representing a *parabola*) interpolated on the extremis and midpoint of the interval, in the points: $f(\alpha)$, $f(\frac{\beta - \alpha}{2})$ and $f(\beta)$.

Considering $I_f(\alpha, \beta) = \int_\alpha^\beta f(x)\,dx$, $\;h_0 = \frac{\beta - \alpha}{2}$ and $P$ the quadratic interpolant of $f$:

$$S_f(\alpha, \beta) = \int_\alpha^\beta P(x)\,dx = \frac{h_0}{3} \Big[f(\alpha) + 4 f(\alpha + h_0) + f(\beta)\Big]$$

This approximation produces an error:

$$E_{Sf}(\alpha, \beta) = I_f(\alpha, \beta) - S_f(\alpha, \beta) = - \frac{h_0^5}{90} f^{(4)}(\xi)$$

where $\xi \in (\alpha, \beta)$.

In [43]:
def simpson_rule(func, alpha, beta):
    """Compute the integrand of the quadratic polynomial of `f' over the interval [alpha, beta].
    
    We use the midpoint of the interval to calculate the stepsize. 
    """
    h_0 = (beta-alpha) * 0.5
    return h_0 * 0.3 * (func(alpha) + 4 * func(alpha + h_0) + func(beta))

In [92]:
func = lambda x: x**2
_a, _b = 0, 1

print(simpson_rule(func, _a, _b))
simpson_rule(func, _a, _b) == 0.3 * 1**3 - 0.3 * 0**2

0.3


True

### Error estimation

To estimate the error $E_{Sf}$ produced when using Simpson's rule, without calculating $f^{(4)}(\xi)$, we can calculate the error $E_{Sf2}$ that would result from computing the definite integral $S_{f2}$ on the two subintervals $[\alpha, \frac{\beta - \alpha}{2}]$ and $[\frac{\beta - \alpha}{2}, \beta]$:

$$S_{f2}(\alpha, \beta) = S_f\Big(\alpha, \frac{\beta - \alpha}{2}\Big) + S_f\Big(\frac{\beta - \alpha}{2}, \beta\Big)$$

$$E_{Sf2}(\alpha, \beta) = \Big(- \frac{h^5}{90} f^{(4)}(\xi)\Big) + \Big(- \frac{h^5}{90} f^{(4)}(\eta)\Big) = - \frac{h^5}{90} \big(f^{(4)}(\xi) + f^{(4)}(\eta)\big)$$

where $\xi \in (\alpha, \frac{\beta - \alpha}{2}), \eta \in (\frac{\beta - \alpha}{2}, \beta)$.

The stepsize $h$ in the two subintervals is $h_0/2$: moreover, if we assume $f$ not to be too much "variable" in the interval $[\alpha, \beta]$, we can approximate $f^{(4)}(\xi) \approx f^{(4)}(\eta)$:

$$E_{Sf2}(\alpha, \beta) \approx - \frac{(h_0/2)^5}{90} \big(2 \cdot f^{(4)}(\xi)\big) = - \frac{h_0^5}{90} \frac{2}{32} \big(f^{(4)}(\xi)\big) = - \frac{h_0^5}{90} \frac{1}{16} \big(f^{(4)}(\xi)\big) \approx I_f(\alpha, \beta) - S_{f2}(\alpha, \beta)$$

$$%E_{Sf2}(\alpha, \beta) - E_{Sf}(\alpha, \beta) \approx $$

Let's call ${\large\varepsilon}_f$ the difference between the two integrations computed with Simpson's rule:

$${\large\varepsilon}_f(\alpha, \beta) = S_f(\alpha, \beta) - S_{f2}(\alpha, \beta)$$

$${\large\varepsilon}_f(\alpha, \beta) \approx \frac{h_0^5}{90} f^{(4)}(\xi) \big(1 - \frac{1}{16}\big) = \frac{h_0^5}{90} \frac{15}{16} f^{(4)}(\xi)$$

$$\frac{{\large\varepsilon}_f(\alpha, \beta)}{15} \approx \frac{h_0^5}{90 \cdot 16} \, f^{(4)}(\xi) \approx I_f(\alpha, \beta) - S_{f2}(\alpha, \beta)$$

The previous equation means that we can calculate the error produced by Simpson's integration rule by computing two more integrations on the two sub-intervals.

In [48]:
def simpson_error(func, alpha, beta, previous_integration=None):

    h_0 = (beta-alpha) * 0.5
    S_f2 = simpson_rule(func, alpha, h_0) + simpson_rule(func, h_0, beta)
    
    if previous_integration is None:
        previous_integration = simpson_rule(func, alpha, beta)
    S_f = previous_integration
    
    epsilon_f = S_f - S_f2
    
    return epsilon_f * 0.1

## Adaptive integration

We now have an estimate of the error committed by the Simpson's numerical integration method on the function $f$, without having to study its fourth derivative. We can overestimate the error produced and constrain it to be below a certain tolerance threshold ${\large\lambda}$:

$$\big|I_f(\alpha, \beta) - S_{f2}(\alpha, \beta)\big| \approx \frac{{\large\varepsilon}_f(\alpha, \beta)}{15} \le \frac{{\large\varepsilon}_f(\alpha, \beta)}{10}\\
\frac{{\large\varepsilon}_f(\alpha, \beta)}{10} \le {\large\lambda} \frac{\beta-\alpha}{b-a}$$

The adaptive integration method computes an approximation of the definite integral of $f$ over the interval $[a, b]$.

The integration is performed by dividing the given interval into smaller intervals, and summing up the integral of those smaller intervals; the criteria used to split the interval uses the previous definition of error of Simpson's rule: if the integral over a subinterval does not satisfy the error estimation, the interval is split in two until the error is contained below a certain threshold.

In [70]:
import numpy as np

lambda_tolerance = 1e-4
# If the integration interval reaches a width below `h_min', stop execution.
h_min = 1e-3

f = lambda x: np.arctan(10 * x)

In [143]:
def adaptive_integration(func, interval, tolerance, h_min, method="simpson"):
    alpha, beta = orig_start, orig_end = interval

    total_sum = 0
    
    # Continue until b reaches the end of the interval of study.
    while abs(orig_end - alpha) > 1e-10:
        # print("{:.4f}".format(alpha), "{:.4f}".format(beta))
        
        if abs(beta - alpha) < h_min or beta < alpha:
            raise ValueError("Attempt to compute the integral on points [{}, {}], which "
                             "are at distance {}, below the minimum, or the end of the "
                             "interval became smaller than the start.".format(alpha, beta, beta - alpha))

        part_integr = simpson_rule(func, alpha, beta)
        error = simpson_error(func, alpha, beta, previous_integration=part_integr)

        if error < tolerance:
            total_sum += part_integr
            alpha, beta = beta, beta + 2 * (beta - alpha)
            if beta > orig_end:
                beta = orig_end
        else:
            beta -= 0.5 * (beta - alpha)

    return total_sum

In [144]:
adaptive_integration(f, (-3, 4), lambda_tolerance, h_min)

1.38719229812847

In [142]:
4 * np.arctan(40) + 3 * np.arctan(-30) - (1/20) * np.log(16/9)

1.5420119327087798